## Team member and Contributions
> Shu Xu (shuxu3@illinois.edu): algorithm development used in System I&II and generated S matrix

> Yan Han (yanhan4@illinois.edu): validity checking of results, and finished the `myICBF` function

> Amrit Kumar(amritk2@illinois.edu): validity checking of results, and developed the web application 

**We finish this notebook together.**

In [1]:
import numpy as np
import pandas as pd

## Read Data

In [2]:
ratings = pd.read_csv("https://liangfgithub.github.io/MovieData/ratings.dat", sep="::", engine='python', header=None)
ratings.columns = ['UserID','MovieID','Rating','Timestamp']
ratings['UserID'] = ratings['UserID'].apply(lambda x:'u' + str(x))
ratings['MovieID'] = ratings['MovieID'].apply(lambda x:'m' + str(x))

In [ ]:
ratings

In [4]:
users = pd.read_csv("https://liangfgithub.github.io/MovieData/users.dat",sep="::",engine='python', encoding="ISO-8859-1", header = None)
users.columns = ['UserID','Gender', 'Age', 'Occupation', 'Zip-code']
users['UserID'] = users['UserID'].apply(lambda x:'m' + str(x))

In [ ]:
users

In [6]:
movies = pd.read_csv("https://liangfgithub.github.io/MovieData/movies.dat",sep="::",engine='python', encoding="ISO-8859-1", header = None)
movies.columns = ['MovieID','Title','Genres']
movies['MovieID'] = movies['MovieID'].apply(lambda x:'m' + str(x))

In [ ]:
movies

## System I: Recommendation Based on Genres
Imagine you know the user’s favorite movie genre. How would you recommend movies to them?

Propose a recommendation scheme along with all the technical details necessary to implement it. For example, you can recommend the **`top five most popular movies`** in that genre, but you need to define what you mean by “most popular.” Similarly, you can recommend the **`top five highly-rated movies`** in that genre, but you must specify how you define “highly-rated.” (Will the movie that receives only one 5-point review be considered highly rated?)

In [236]:
grouped_ratings = ratings.groupby('MovieID').agg({'Rating': ['count', 'mean'] })
grouped_ratings.columns = grouped_ratings.columns.droplevel(0)
grouped_ratings.reset_index(inplace=True)
grouped_ratings.rename(columns = {'count': 'RatingFreq', 'mean': 'RatingAvg'}, inplace=True)

In [237]:
genre_list = ["Action", "Adventure", "Animation","Children's", "Comedy", 
              "Crime","Documentary", "Drama", "Fantasy","Film-Noir", 
              "Horror", "Musical",  "Mystery", "Romance", "Sci-Fi",
               "Thriller", "War", "Western"]

In [238]:
movies_ratings = movies.merge(grouped_ratings, left_on='MovieID', right_on='MovieID')

In [239]:
movies_ratings

,MovieID,Title,Genres,RatingFreq,RatingAvg
0,m1,Toy Story (1995),Animation|Children's|Comedy,2077,4.146846
1,m2,Jumanji (1995),Adventure|Children's|Fantasy,701,3.201141
2,m3,Grumpier Old Men (1995),Comedy|Romance,478,3.016736
3,m4,Waiting to Exhale (1995),Comedy|Drama,170,2.729412
4,m5,Father of the Bride Part II (1995),Comedy,296,3.006757
...,...,...,...,...,...
3701,m3948,Meet the Parents (2000),Comedy,862,3.635731
3702,m3949,Requiem for a Dream (2000),Drama,304,4.115132
3703,m3950,Tigerland (2000),Drama,54,3.666667
3704,m3951,Two Family House (2000),Drama,40,3.900000


### Method 1 Top Five most Popular Movies

In [240]:
genre_movie_id_rating_freq = {}
genre_movie_title_rating_freq = {}
K = 5  # top 5
for genre in genre_list:
    true_table = movies_ratings['Genres'].apply(lambda x: True if genre in x else False)
    rating_freq_df = movies_ratings[true_table].sort_values('RatingFreq', ascending=False)
    genre_movie_id_rating_freq[genre] = rating_freq_df['MovieID'].values[:K]
    genre_movie_title_rating_freq[genre] = rating_freq_df['Title'].values[:K]

top5_rating_freq_with_movie_id = pd.DataFrame.from_dict(genre_movie_id_rating_freq).T
top5_rating_freq_with_movie_title = pd.DataFrame.from_dict(genre_movie_title_rating_freq).T 

In [241]:
top5_rating_freq_with_movie_id

,0,1,2,3,4
Action,m260,m1196,m1210,m480,m2028
Adventure,m260,m1196,m1210,m480,m1580
Animation,m1,m2987,m2355,m3114,m588
Children's,m1097,m1,m34,m919,m2355
Comedy,m2858,m1270,m1580,m2396,m1197
Crime,m608,m1617,m858,m296,m50
Documentary,m2064,m246,m162,m3007,m1147
Drama,m2858,m1196,m2028,m593,m608
Fantasy,m260,m1097,m2628,m2174,m2797
Film-Noir,m1617,m541,m2987,m1252,m913


In [242]:
top5_rating_freq_with_movie_title

,0,1,2,3,4
Action,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode VI - Return of the Jedi (1983),Jurassic Park (1993),Saving Private Ryan (1998)
Adventure,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode V - The Empire Strikes Back...,Star Wars: Episode VI - Return of the Jedi (1983),Jurassic Park (1993),Men in Black (1997)
Animation,Toy Story (1995),Who Framed Roger Rabbit? (1988),"Bug's Life, A (1998)",Toy Story 2 (1999),Aladdin (1992)
Children's,E.T. the Extra-Terrestrial (1982),Toy Story (1995),Babe (1995),"Wizard of Oz, The (1939)","Bug's Life, A (1998)"
Comedy,American Beauty (1999),Back to the Future (1985),Men in Black (1997),Shakespeare in Love (1998),"Princess Bride, The (1987)"
Crime,Fargo (1996),L.A. Confidential (1997),"Godfather, The (1972)",Pulp Fiction (1994),"Usual Suspects, The (1995)"
Documentary,Roger & Me (1989),Hoop Dreams (1994),Crumb (1994),American Movie (1999),When We Were Kings (1996)
Drama,American Beauty (1999),Star Wars: Episode V - The Empire Strikes Back...,Saving Private Ryan (1998),"Silence of the Lambs, The (1991)",Fargo (1996)
Fantasy,Star Wars: Episode IV - A New Hope (1977),E.T. the Extra-Terrestrial (1982),Star Wars: Episode I - The Phantom Menace (1999),Beetlejuice (1988),Big (1988)
Film-Noir,L.A. Confidential (1997),Blade Runner (1982),Who Framed Roger Rabbit? (1988),Chinatown (1974),"Maltese Falcon, The (1941)"


### Method 2 Top Five high-rated Movies

In [97]:
genre_movie_id_rating_avg = {}
genre_movie_title_rating_avg = {}
K = 5  # top 5
for genre in genre_list:
    true_table = movies_ratings['Genres'].apply(lambda x: True if genre in x else False)
    rating_avg_df = movies_ratings[true_table].sort_values('RatingAvg', ascending=False)
    genre_movie_id_rating_avg[genre] = rating_avg_df['MovieID'].values[:K]
    genre_movie_title_rating_avg[genre] = rating_avg_df['Title'].values[:K]

top5_rating_avg_with_movie_id = pd.DataFrame.from_dict(genre_movie_id_rating_avg).T
top5_rating_avg_with_movie_title = pd.DataFrame.from_dict(genre_movie_title_rating_avg).T 

In [98]:
top5_rating_avg_with_movie_id

,0,1,2,3,4
Action,m2905,m2019,m858,m1198,m260
Adventure,m3172,m2905,m1198,m260,m1204
Animation,m745,m1148,m720,m1223,m3429
Children's,m919,m3114,m1,m2761,m1023
Comedy,m3233,m1830,m3607,m745,m1148
Crime,m3656,m858,m50,m3517,m3435
Documentary,m3881,m787,m3338,m2930,m128
Drama,m3382,m989,m3607,m3245,m53
Fantasy,m260,m792,m1097,m247,m1073
Film-Noir,m922,m3435,m913,m1252,m1267


In [99]:
top5_rating_avg_with_movie_title

,0,1,2,3,4
Action,Sanjuro (1962),Seven Samurai (The Magnificent Seven) (Shichin...,"Godfather, The (1972)",Raiders of the Lost Ark (1981),Star Wars: Episode IV - A New Hope (1977)
Adventure,Ulysses (Ulisse) (1954),Sanjuro (1962),Raiders of the Lost Ark (1981),Star Wars: Episode IV - A New Hope (1977),Lawrence of Arabia (1962)
Animation,"Close Shave, A (1995)","Wrong Trousers, The (1993)",Wallace & Gromit: The Best of Aardman Animatio...,"Grand Day Out, A (1992)",Creature Comforts (1990)
Children's,"Wizard of Oz, The (1939)",Toy Story 2 (1999),Toy Story (1995),"Iron Giant, The (1999)",Winnie the Pooh and the Blustery Day (1968)
Comedy,Smashing Time (1967),Follow the Bitch (1998),One Little Indian (1973),"Close Shave, A (1995)","Wrong Trousers, The (1993)"
Crime,Lured (1947),"Godfather, The (1972)","Usual Suspects, The (1995)","Bells, The (1926)",Double Indemnity (1944)
Documentary,Bittersweet Motel (2000),"Gate of Heavenly Peace, The (1995)",For All Mankind (1989),Return with Honor (1998),Jupiter's Wife (1994)
Drama,Song of Freedom (1936),Schlafes Bruder (Brother of Sleep) (1995),One Little Indian (1973),I Am Cuba (Soy Cuba/Ya Kuba) (1964),Lamerica (1994)
Fantasy,Star Wars: Episode IV - A New Hope (1977),"Hungarian Fairy Tale, A (1987)",E.T. the Extra-Terrestrial (1982),Heavenly Creatures (1994),Willy Wonka and the Chocolate Factory (1971)
Film-Noir,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Double Indemnity (1944),"Maltese Falcon, The (1941)",Chinatown (1974),"Manchurian Candidate, The (1962)"


## System II: Recommendation Based on Item-Based Collaborative Filtering(IBCF)
For this system, follow these steps. Let `R` denote the 6040-by-3706 rating matrix.
1. Normalize the rating matrix by centering each row. This means subtracting row means from each row of the rating matrix `R`. Row means should be computed based on non-NA entries. For instance, the mean of a vector like `(2, 4, NA, NA)` should be `3`.
2. Compute the Cosine similarity among the `3,706` movies. For movies $i$ and $j$, let $\mathcal{I}_{ij}$ denote the set of users who rated both movies $i$ and $j$. We decide to ignore similarities computed based on less than three user ratings. Thus, define the similarity between movie $i$ and movie $j$ as follows, when the cardinality of $\mathcal{I}_{ij}$ is bigger than two,
$$
    S_{ij} = \frac{1}{2} + \frac{1}{2} \frac{\sum_{l \in \mathcal{I}_{ij}} R_{li} R_{lj}}{\sqrt{\sum_{l \in \mathcal{I}_{ij}} R^2_{li}} \sqrt{\sum_{l \in \mathcal{I}_{ij}} R^2_{lj}}}
$$

This transformation $(1 + cos)/2$ ensures that similarity measures are between 0 and 1. NA values may occur when 1) the set $\mathcal{I}_{ij}$ has a cardinality less than or equal to two (i.e., this pair of movies have been rated by only zero, one, or two users) or 2) one of the denominators is zero.

3. Let $S$ denote the `3706-by-3706` similarity matrix computed in previous step. For each row, sort the non-NA similarity measures and keep the top 30, setting the rest to NA. This new similarity matrix, still denoted as $S$, is no longer symmetric. Save this matrix online. 

Display the pairwise similarity values from the S matrix for the following specified movies: `“m1”, “m10”, “m100”, “m1510”, “m260”, “m3212”`. Please round the results to `7` decimal places.

4. Create a function named `myIBCF`:
- **Input**: `newuser`, a `3706-by-1` vector (denoted as $w$) containing ratings for the `3,706` movies from a new user. Many entries in this vector will be zero. The order of the movies in this vector should match the rating matrix $R$. (Should we center $w$? For IBCF, centering the new user ratings is not necessary.)

- **Inside the function**: Upon receiving this input, your function should download the similarity matrix and use it to compute predictions for movies that have not been rated by this new user yet. Use the following formula to compute the prediction for movie $l$:
$$
    \frac{1}{\sum_{i \in S(l)}S_{li}}{\sum \limits_{i \in S(l)} S_{li} w_i}
$$

where $S(l)$ denotes the set of movies in the 30-nearest neighborhood of movie $l$. Again NA values may occur.

- **Output**: Based on your predictions, recommend the `top 10 movies` to this new user, using the column names of the rating matrix R. Explain what your code should do if fewer than 10 predictions are non-NA. Provide a method to suggest additional movies that have not been rated by this user.


**Test your function**

For your function `myIBCF`, print the `top 10 recommendations` for the following three users:
- User “u1181” from the rating matrix $R$
- User “u1351” from the rating matrix $R$
- A hypothetical user who rates movie “m1613” with 5 and movie “m1755” with 4.

In [313]:
movie_rating_2d_df = pd.read_csv("Movie_Rmat.csv")

In [314]:
movie_rating_2d_df

,m1,m10,m100,m1000,m1002,m1003,m1004,m1005,m1006,m1007,...,m99,m990,m991,m992,m993,m994,m996,m997,m998,m999
u1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u10,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u1000,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u1001,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
u995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u996,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
u997,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
# Step 1: Normalize the rating matrix by centering each row
movie_rating_2d_normalized_df = movie_rating_2d_df.sub(movie_rating_2d_df.mean(axis=1, skipna=True), axis=0)

In [310]:
# Step 2: Compute cosine similarity between users for a given movie
def cosine_similarity(a: np.array,
                      b: np.array,
                      threshold: int = 3):
    masked_a = np.ma.array(a, mask=np.isnan(a)) # Use a mask to mark the NaNs; mask=True (NaN)
    masked_b = np.ma.array(b, mask=np.isnan(b))
    combined_nan_mask = masked_a.mask | masked_b.mask
    # Ignore similiarity if 
    # (1) the movie few than 3 ratings    
    if ((~combined_nan_mask).sum() < threshold):
        return np.nan
    a_valid_norm = np.sqrt((np.ma.array(a, mask=combined_nan_mask)**2).sum())
    b_valid_norm = np.sqrt((np.ma.array(b, mask=combined_nan_mask)**2).sum())
    # Ignore similiarity if 
    # (2) a_valid_norm or b_valid_norm is 0
    if (a_valid_norm > 0.0) and (b_valid_norm > 0.0):
        return (1.0 + np.ma.dot(masked_a, masked_b)/(a_valid_norm * b_valid_norm))/2
    else:
        return np.nan

In [311]:
movie_ids = movie_rating_2d_normalized_df.columns
n_movies = len(movie_ids)
S = np.ones((n_movies, n_movies)) * np.nan

In [ ]:
for i in range(n_movies):
    #print(i)
    for j in range(i+1, n_movies):
        cosine_similarity_val = cosine_similarity(movie_rating_2d_normalized_df.iloc[:, i].values, movie_rating_2d_normalized_df.iloc[:, j].values)
        S[i][j] = cosine_similarity_val
        S[j][i] = cosine_similarity_val

In [380]:
report_movie_ids_list = ['m1', 'm10', 'm100', 'm1510', 'm260', 'm3212']
report_index_list = movie_ids.get_indexer(report_movie_ids_list)
S_report_dict = {}
for movie_index in report_index_list:
    S_report_dict[movie_ids[movie_index]] = S[movie_index, report_index_list]
S_report_df = pd.DataFrame.from_dict(S_report_dict)
S_report_df.index = report_movie_ids_list     

In [381]:
S_report_df

,m1,m10,m100,m1510,m260,m3212
m1,NaN,0.512106,0.392000,NaN,0.741148,NaN
m10,0.512106,NaN,0.547458,NaN,0.534334,NaN
m100,0.392000,0.547458,NaN,NaN,0.329694,NaN
m1510,NaN,NaN,NaN,NaN,NaN,NaN
m260,0.741148,0.534334,0.329694,NaN,NaN,NaN
m3212,NaN,NaN,NaN,NaN,NaN,NaN


In [377]:
# 3. For each row, sort the non-NA similarity measures and keep the top 30, setting the rest to NA. 
# This new similarity matrix, still denoted as $S$, is no longer symmetric. Save this matrix online.
K = 30
new_S = np.nan_to_num(S, copy=True, nan=0.0)   # replace nan with 0.0, otherise NaN will be list in the end

In [378]:
for i in range(n_movies):
    temp_nan_mask = np.ones(n_movies, dtype=bool)
    top_K_indices = np.argsort(new_S[i, :])[-K:]  # in ascending order
    temp_nan_mask[top_K_indices] = False   # protect the top K
    new_S[i, temp_nan_mask] = np.nan   # set the rest to NA

In [388]:
new_S_df = pd.DataFrame(new_S, columns=movie_ids, index=movie_ids)

In [391]:
new_S_df.to_csv('S.csv', na_rep='NA')

### Step 4: myIBCF function

In [10]:
S = pd.read_csv('S.csv', index_col=0)

In [134]:
S.head()

,m1,m10,m100,m1000,m1002,m1003,m1004,m1005,m1006,m1007,...,m99,m990,m991,m992,m993,m994,m996,m997,m998,m999
m1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [349]:
def myIBCF(newuser, S, popular_id_100=popular_id_100, all_movie_ids=all_movie_ids, top_n=10):
    if isinstance(newuser, dict):
        # create newuser from dict
        rated_movies = set(newuser.keys())
        newuser = pd.Series(newuser, index=all_movie_ids)
    else:
        rated_movies = set(newuser.loc[~pd.isna(newuser)].index.values)
        
    # generate rating predictions
    prediction = S.multiply(newuser, axis=1).sum(axis=1) / S.loc[:, ~pd.isna(newuser)].sum(axis=1)
    sorted_pred = prediction[prediction != float('inf')].sort_values(ascending=False)
    
    # uncomment the line to see predicted ratings
#     print(sorted_pred[~sorted_pred.isna()][:20])
    
    # remove invalid values and movies already rated by user
    recommend_id = list(sorted_pred[~sorted_pred.isna()].index.values)
    recommend_id = [item for item in recommend_id if item not in rated_movies]
    
    # if less than top_n recommendations, add most popular movies that are not rated by the user 
    # and not in the recommendation list
    i = 0
    while len(recommend_id) < top_n and i < 100:
        if popular_id_100[i] not in rated_movies and popular_id_100[i] not in recommend_id:
            recommend_id.append(popular_id_100[i])
        i += 1
    
    return recommend_id[:top_n]

In [350]:
newuser = movie_rating_2d_df.loc['u1181']
myIBCF(newuser, S)

['m3732',
 'm749',
 'm3899',
 'm337',
 'm1734',
 'm1914',
 'm1253',
 'm504',
 'm1235',
 'm3752']

In [351]:
newuser = movie_rating_2d_df.loc['u1351']
myIBCF(newuser, S)

['m1871',
 'm1901',
 'm1514',
 'm2061',
 'm2127',
 'm2858',
 'm1877',
 'm2000',
 'm2028',
 'm2063']

In [352]:
# list of all movie IDs, used to construct newuser from dict
all_movie_ids = movie_rating_2d_df.columns

In [353]:
# recommend most popular movies if less than 10 recommendations generated
# the list can be hardcoded
popular_id_100 = movies_ratings.sort_values("RatingFreq", ascending=False)["MovieID"][:100].values

In [354]:
popular_id_100

array(['m2858', 'm260', 'm1196', 'm1210', 'm480', 'm2028', 'm589',
       'm2571', 'm1270', 'm593', 'm1580', 'm1198', 'm608', 'm2762',
       'm110', 'm2396', 'm1197', 'm527', 'm1617', 'm1265', 'm1097',
       'm2628', 'm2997', 'm318', 'm858', 'm356', 'm2716', 'm296', 'm1240',
       'm1', 'm1214', 'm2916', 'm457', 'm3578', 'm1200', 'm541', 'm2987',
       'm1259', 'm50', 'm34', 'm2791', 'm780', 'm3175', 'm1193', 'm919',
       'm924', 'm1127', 'm2355', 'm1387', 'm1221', 'm912', 'm1036',
       'm1213', 'm1610', 'm377', 'm1291', 'm2000', 'm1136', 'm3114',
       'm1307', 'm1704', 'm1721', 'm1968', 'm648', 'm2599', 'm3793',
       'm32', 'm2174', 'm2797', 'm2291', 'm2918', 'm3471', 'm590',
       'm2959', 'm1374', 'm2683', 'm1394', 'm592', 'm1784', 'm1573',
       'm1304', 'm3418', 'm223', 'm380', 'm2706', 'm1225', 'm1584',
       'm1527', 'm3481', 'm1923', 'm750', 'm2699', 'm39', 'm21', 'm1393',
       'm2804', 'm588', 'm2406', 'm1220', 'm733'], dtype=object)

In [355]:
myIBCF({"m1613": 5, "m1755": 4}, S)

['m1017',
 'm765',
 'm3269',
 'm74',
 'm691',
 'm2805',
 'm2870',
 'm340',
 'm338',
 'm3258']

In [356]:
# newuser with less than 10 recommendations
myIBCF({"m161": 5}, S, popular_id_100, all_movie_ids)

['m844',
 'm2858',
 'm260',
 'm1196',
 'm1210',
 'm480',
 'm2028',
 'm589',
 'm2571',
 'm1270']